# Data structure interface and matrix algebra examples
Felix Zaussinger | 13.11.2020

## Core Analysis Goal(s)
1. Settle for a data structure that can well represent the playing field at all times of the game.
2. Find a suitable data structure for defining the "plots" or "blocks" on which
players can manage their land
3. Define the interface: what are the dimensions (x, y, z)

## Key Insight(s)
1. Numpy masked arrays (np.ma) are perfectly fine. We should use those.
2. A simple numpy function (np.block) may be enough to work with these blocks

## Explanations
1. Matrix Algebra >> Python for-loops
2. Python itself can be quite slow, which is why numpy is written in a lower-level
language (C). If you use the numpy matrix computations, for-loops still need to be
used to calculate things, but they will be executed in C, which is MUCH faster.
This is the crucial difference. Does not matter much for 80 x 80 matrix,
but I would wish to not close the door to being able to the game on a 800 x 800
matrix as well! (:

In [8]:
import os
import numpy as np
import numpy.ma as ma

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("ticks")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Define matrix size

In [21]:
n_plots = 4
rows = cols = 80

# check if playing field size and plot number works together
try:
    # modulo must be zero
    assert rows % n_plots == 0
except AssertionError as e:
    print("playing field size and plot number dont work together. field size must be divideable by plot number without rest.")

Create a dummy block matrix (suggestions for a different, maybe clearer synonym for plot)

In [29]:
block_length = int(rows/n_plots)
block_dummy = np.ones(block_length**2).reshape((block_length, block_length))
block_dummy.shape

(20, 20)

Create a few meaningful block matrixes

Example matrix:

  1 2

A x y

B z x

In [30]:
A1 = block_dummy
A2 = block_dummy * 2
B1 = block_dummy * 3
B2 = block_dummy * 4

Create a bigger (40 x 40) matrix from the 4 smaller block matrices

In [31]:
large_matrix = np.block([[A1, A2], [B1, B2]])
large_matrix

array([[1., 1., 1., ..., 2., 2., 2.],
       [1., 1., 1., ..., 2., 2., 2.],
       [1., 1., 1., ..., 2., 2., 2.],
       ...,
       [3., 3., 3., ..., 4., 4., 4.],
       [3., 3., 3., ..., 4., 4., 4.],
       [3., 3., 3., ..., 4., 4., 4.]])

In [32]:
large_matrix.shape


(40, 40)